#Face detection

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/face-mask-detection")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'face-mask-detection' dataset.
Path to dataset files: /kaggle/input/face-mask-detection


In [2]:
pip install opendatasets scikit-learn pyyaml pandas

In [3]:
import os
import shutil
import glob
import random
import xml.etree.ElementTree as ET
import yaml
from sklearn.model_selection import train_test_split
import opendatasets as od # Ensure this is imported for the download function

# --- Configuration ---
DATASET_URL = 'https://www.kaggle.com/datasets/andrewmvd/face-mask-detection'
OUTPUT_DIR = 'datasets/face_mask_yolo_clean'

# We ONLY define the classes you want.
# Any class in the XML not in this list is automatically ignored.
CLASSES = ["with_mask", "without_mask"]

def download_dataset():
    """Downloads dataset using opendatasets library."""
    if os.path.exists('face-mask-detection'):
        print("Dataset folder found. Skipping download.")
        return

    print(f"Downloading dataset from {DATASET_URL}...")
    try:
        od.download(DATASET_URL, data_dir='.')
    except Exception as e:
        print(f"Error downloading: {e}")
        print("Please ensure you have your kaggle.json file or credentials ready.")
        exit()

def convert_box(size, box):
    """Converts (xmin, ymin, xmax, ymax) to YOLO (x_center, y_center, width, height)."""
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    return (x * dw, y * dh, w * dw, h * dh)

def convert_xml_to_txt(xml_path, output_path):
    """Parses XML and writes YOLO format .txt file, filtering unwanted classes."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        yolo_lines = []

        for obj in root.iter('object'):
            cls = obj.find('name').text

            # --- FILTERING LOGIC ---
            # If the class found in XML is not in our desired CLASSES list, skip it.
            if cls not in CLASSES:
                continue

            cls_id = CLASSES.index(cls)

            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                 float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))

            bb = convert_box((w, h), b)
            yolo_lines.append(f"{cls_id} {bb[0]:.6f} {bb[1]:.6f} {bb[2]:.6f} {bb[3]:.6f}")

        # Only create the file if we found valid classes (with_mask or without_mask)
        if yolo_lines:
            with open(output_path, 'w') as f:
                f.write('\n'.join(yolo_lines))
            return True # Success: Image contains relevant data
        else:
            return False # Fail: Image only contained "incorrect" masks or nothing

    except Exception as e:
        print(f"Warning: Issue parsing {xml_path}: {e}")
        return False

def prepare_structure():
    """Creates the YOLO directory structure."""
    dirs = [
        f'{OUTPUT_DIR}/images/train',
        f'{OUTPUT_DIR}/images/val',
        f'{OUTPUT_DIR}/labels/train',
        f'{OUTPUT_DIR}/labels/val'
    ]
    for d in dirs:
        os.makedirs(d, exist_ok=True)
    # Clear existing data.yaml if it exists to avoid confusion
    if os.path.exists(f'{OUTPUT_DIR}/data.yaml'):
        os.remove(f'{OUTPUT_DIR}/data.yaml')

def process_data():
    # 1. Download
    download_dataset()

    # 2. Setup Directories
    prepare_structure()

    # 3. Get all images
    source_img_dir = 'face-mask-detection/images'
    source_xml_dir = 'face-mask-detection/annotations'

    images = glob.glob(os.path.join(source_img_dir, '*.png'))
    if not images:
        print("No images found! Check if the dataset downloaded correctly.")
        return

    print(f"Found {len(images)} total source images.")

    # 4. Split Train/Val
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    def process_batch(image_list, split_type):
        count = 0
        for img_path in image_list:
            filename = os.path.basename(img_path)
            xml_filename = filename.replace('.png', '.xml')
            xml_path = os.path.join(source_xml_dir, xml_filename)

            target_img_path = f'{OUTPUT_DIR}/images/{split_type}/{filename}'
            target_label_path = f'{OUTPUT_DIR}/labels/{split_type}/{filename.replace(".png", ".txt")}'

            if os.path.exists(xml_path):
                # Only copy image if convert_xml_to_txt returns True (meaning it has valid labels)
                has_valid_labels = convert_xml_to_txt(xml_path, target_label_path)
                if has_valid_labels:
                    shutil.copy(img_path, target_img_path)
                    count += 1
        return count

    print("Processing Training Set...")
    train_count = process_batch(train_imgs, 'train')
    print("Processing Validation Set...")
    val_count = process_batch(val_imgs, 'val')

    print(f"Done! Included {train_count + val_count} images (excluded images with only 'incorrect' masks).")

def create_yaml():
    """Generates the data.yaml file."""
    yaml_content = {
        'path': os.path.abspath(OUTPUT_DIR),
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(CLASSES),
        'names': CLASSES
    }

    yaml_path = f'{OUTPUT_DIR}/data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)
    print(f"Generated config file at: {yaml_path}")

if __name__ == "__main__":
    process_data()
    create_yaml()
    print("\nREADY TO TRAIN!")
    print(f"Config file: {os.path.abspath(OUTPUT_DIR)}/data.yaml")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: AlexandruSauca
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection


100%|██████████| 398M/398M [00:00<00:00, 1.67GB/s]



Found 853 total source images.
Processing Training Set...
Processing Validation Set...
Done! Included 832 images (excluded images with only 'incorrect' masks).
Generated config file at: datasets/face_mask_yolo_clean/data.yaml

READY TO TRAIN!
Config file: /content/datasets/face_mask_yolo_clean/data.yaml


In [4]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00


In [6]:
!yolo task=detect mode=train model=yolov8n.pt data='/content/datasets/face_mask_yolo_clean/data.yaml' epochs=200 imgsz=640 hsv_v=0.4 hsv_s=0.7 scale=0.8 degrees=15.0 mosaic=0.5 mixup=0.1

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/face_mask_yolo_clean/data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgs

In [8]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/best.pt")
model_onnx = model.export(format='onnx')

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<=1.19.1', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 139ms
Prepared 6 packages in 3.55s
Installed 6 packages in 263ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.19.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.78

requirements: AutoUpdate success ✅ 4.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting 

In [5]:
!unzip /content/drive/MyDrive/dataset.zip

Archive:  /content/drive/MyDrive/dataset.zip
   creating: dataset/
   creating: dataset/db/
   creating: dataset/db/indoor_persons/
  inflating: dataset/db/indoor_persons/Andres - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Cristina - Indoor.jpg  
  inflating: dataset/db/indoor_persons/DaniB - Indoor.jpg  
  inflating: dataset/db/indoor_persons/DaniF - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Diego - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Ester - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Isa - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Marcos - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Narciso - Indoor.jpg  
  inflating: dataset/db/indoor_persons/Pablo - Indoor.jpg  
   creating: dataset/db/outdoor_persons/
  inflating: dataset/db/outdoor_persons/Andres - Outdoor.jpg  
  inflating: dataset/db/outdoor_persons/Cristina - Outdoor.jpg  
  inflating: dataset/db/outdoor_persons/DaniB - Outdoor.jpg  
  inflating: dataset/db/outdoor_

In [1]:
import os
import glob
from ultralytics import YOLO
from IPython.display import Image, display
import random

# --- CONFIGURATION ---
# The root folder containing 'Indoor', 'Outdoor', etc.
TEST_ROOT_PATH = '/content/dataset/test'
MODEL_PATH = '/content/runs/detect/train/weights/best.pt'

# Extensions to look for
EXTENSIONS = ['*.jpg', '*.jpeg', '*.png', '*.bmp']

def test_recursive():
    # 1. Check Model
    if not os.path.exists(MODEL_PATH):
        print(f"Error: Model not found at {MODEL_PATH}")
        return

    # 2. Find ALL images recursively (Indoor, Outdoor, masked, etc.)
    image_paths = []
    for ext in EXTENSIONS:
        # ** matches any subfolder level
        found = glob.glob(os.path.join(TEST_ROOT_PATH, '**', ext), recursive=True)
        image_paths.extend(found)

    if not image_paths:
        print(f"No images found in {TEST_ROOT_PATH}. Check your path or file extensions.")
        return

    print(f"Found {len(image_paths)} images in subfolders.")
    print(f"Loading model from {MODEL_PATH}...")
    model = YOLO(MODEL_PATH)

    # 3. Run Inference on the list of images
    # We pass the list 'image_paths' directly to source
    print("Running inference...")
    results = model.predict(source=image_paths, save=True, conf=0.5, save_txt=False, stream=False)

    print("\n" + "="*40)
    print("TESTING COMPLETE")
    print("="*40)

    # 4. Locate Results
    # Find the latest prediction folder
    predict_dirs = glob.glob('/content/runs/detect/predict*')
    latest_dir = max(predict_dirs, key=os.path.getmtime)
    print(f"All detection results saved to: {latest_dir}")

    # 5. Display 3 Random Samples
    print("\nDisplaying random results:")
    result_images = glob.glob(os.path.join(latest_dir, '*'))

    # Pick 3 random images to show diversity
    if len(result_images) > 0:
        samples = random.sample(result_images, min(3, len(result_images)))
        for img_path in samples:
            display(Image(filename=img_path, width=400))
            print(f"Shown: {os.path.basename(img_path)}")

if __name__ == "__main__":
    test_recursive()

Found 405 images in subfolders.
Loading model from /content/runs/detect/train/weights/best.onnx...
Running inference...
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})



RuntimeError: Error in execution: Got invalid dimensions for input: images for the following indices
 index: 0 Got: 405 Expected: 1
 Please fix either the inputs/outputs or the model.

In [2]:
import os
import glob
from ultralytics import YOLO
from IPython.display import Image, display
import random

# --- CONFIGURATION ---
TEST_ROOT_PATH = '/content/dataset/test'
# Keep ONNX if you really need it
MODEL_PATH = '/content/runs/detect/train/weights/best.onnx'
EXTENSIONS = ['*.jpg', '*.jpeg', '*.png', '*.bmp']

def test_recursive_onnx():
    if not os.path.exists(MODEL_PATH):
        print(f"Error: Model not found at {MODEL_PATH}")
        return

    # 1. Find images
    image_paths = []
    for ext in EXTENSIONS:
        found = glob.glob(os.path.join(TEST_ROOT_PATH, '**', ext), recursive=True)
        image_paths.extend(found)

    if not image_paths:
        print("No images found.")
        return

    print(f"Found {len(image_paths)} images. Loading ONNX model...")
    model = YOLO(MODEL_PATH)

    print("Running inference (One-by-One for ONNX)...")

    # --- THE FIX FOR ONNX ---
    # We loop through the images and predict one at a time
    for i, img_path in enumerate(image_paths):
        # We process one image at a time to satisfy "Expected: 1"
        # stream=False ensures it finishes before moving on
        model.predict(source=img_path, save=True, conf=0.5, save_txt=False)

        # Optional: Print progress every 10 images
        if i % 10 == 0:
            print(f"Processed {i}/{len(image_paths)}...")

    print("\nTESTING COMPLETE")

    # Display logic remains similar...
    predict_dirs = glob.glob('/content/runs/detect/predict*')
    latest_dir = max(predict_dirs, key=os.path.getmtime)
    print(f"Results saved to: {latest_dir}")

    result_images = glob.glob(os.path.join(latest_dir, '*'))
    if len(result_images) > 0:
        samples = random.sample(result_images, min(3, len(result_images)))
        for img_path in samples:
            display(Image(filename=img_path, width=400))
            print(f"Shown: {os.path.basename(img_path)}")

if __name__ == "__main__":
    test_recursive_onnx()

Output hidden; open in https://colab.research.google.com to view.

In [9]:
pip install sahi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88


In [2]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# 1. Load model through SAHI wrapper
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='/content/runs/detect/train/weights/best.onnx',
    confidence_threshold=0.35,
    device="cuda"  # or 'cpu'
)

# 2. Run "Sliced" Prediction
result = get_sliced_prediction(
    "/content/dataset/test/Indoor/Non-masked/Andres - Indoor - 21C.png",
    detection_model,
    slice_height=320,
    slice_width=320,
    overlap_height_ratio=0.2, # Overlap ensures faces on the edge of a slice aren't missed
    overlap_width_ratio=0.2
)

result.export_visuals(export_dir="results/")

Performing prediction on 135 slices.


In [3]:
import os
import shutil
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# --- CONFIGURATION ---
SOURCE_ROOT = "/content/dataset/test"  # Where your images are
OUTPUT_ROOT = "/content/results"       # Where to save results
MODEL_PATH = "/content/runs/detect/train/weights/best.pt"

# 1. Load Model (Once)
print(f"Loading model from {MODEL_PATH}...")
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=MODEL_PATH,
    confidence_threshold=0.5,
    device="cuda"  # Change to 'cpu' if you don't have a GPU
)

# 2. Iterate recursively through all folders
print(f"Processing images from {SOURCE_ROOT}...")

for root, dirs, files in os.walk(SOURCE_ROOT):
    for filename in files:
        # Filter for images only
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):

            # Construct full file path
            source_path = os.path.join(root, filename)

            # Create corresponding output path (Mirror structure)
            # e.g., /content/dataset/test/Indoor/masked -> /content/results/Indoor/masked
            relative_path = os.path.relpath(root, SOURCE_ROOT)
            target_folder = os.path.join(OUTPUT_ROOT, relative_path)
            os.makedirs(target_folder, exist_ok=True)

            print(f"Predicting: {filename}")

            try:
                # 3. Run "Sliced" Prediction
                result = get_sliced_prediction(
                    source_path,
                    detection_model,
                    slice_height=320,
                    slice_width=320,
                    overlap_height_ratio=0.2,
                    overlap_width_ratio=0.2
                )

                # 4. Export Result
                # export_visuals saves a file named "prediction_visual.png"
                result.export_visuals(export_dir=target_folder, file_name="temp_result")

                # 5. Rename to original filename (so we don't overwrite)
                # SAHI exports as [file_name].png, usually.
                generated_file = os.path.join(target_folder, "temp_result.png")
                final_path = os.path.join(target_folder, filename)

                if os.path.exists(generated_file):
                    # If source was .jpg and sahi saved .png, this handles the extension switch
                    if os.path.exists(final_path):
                        os.remove(final_path) # Overwrite if exists
                    os.rename(generated_file, final_path)

            except Exception as e:
                print(f"Error processing {filename}: {e}")

print("\n" + "="*40)
print(f"Done! Results saved to: {OUTPUT_ROOT}")
print("="*40)

Loading model from /content/runs/detect/train/weights/best.onnx...
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Processing images from /content/dataset/test...
Predicting: Andres - Outdoor - 24C.png
Performing prediction on 135 slices.
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Predicting: Pablo - Outdoor - 6C.png
Performing prediction on 135 slices.
Predicting: Cristina - Outdoor - 12C.png
Performing prediction on 135 slices.
Predicting: Narciso - Outdoor - 6C.png
Perfor

KeyboardInterrupt: 

In [5]:
import os
import shutil
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# --- CONFIGURATION ---
SOURCE_ROOT = "/content/dataset/test"  # Input folder
OUTPUT_ROOT = "/content/results"       # Output folder
MODEL_PATH = "/content/runs/detect/train/weights/best.onnx" # Your model

# 1. Load Model
print(f"Loading model from {MODEL_PATH}...")
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=MODEL_PATH,
    confidence_threshold=0.5,
    device="cuda" # Change to 'cpu' if needed
)

# 2. Iterate recursively through all folders
print(f"Processing images from {SOURCE_ROOT} (Max 10 per folder)...")

total_count = 0
errors = 0

for root, dirs, files in os.walk(SOURCE_ROOT):
    # Optional: Sort files to ensure we always pick the same "first 10"
    files.sort()

    # --- NEW: Counter for the current subfolder ---
    folder_image_count = 0

    for filename in files:
        # Filter for images
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):

            # --- NEW: Check limit ---
            if folder_image_count >= 10:
                # We hit the limit for this specific folder.
                # Break the inner loop to move to the NEXT folder in os.walk
                break

            source_path = os.path.join(root, filename)

            # Create corresponding output path (Mirror structure)
            relative_path = os.path.relpath(root, SOURCE_ROOT)
            target_folder = os.path.join(OUTPUT_ROOT, relative_path)
            os.makedirs(target_folder, exist_ok=True)

            # Simple progress print
            print(f"[{total_count+1}] Processing: {filename} ... ", end="")

            try:
                # 3. Run "Sliced" Prediction
                result = get_sliced_prediction(
                    source_path,
                    detection_model,
                    slice_height=320,
                    slice_width=320,
                    overlap_height_ratio=0.2,
                    overlap_width_ratio=0.2
                )

                # 4. Export Result (Saves to disk)
                result.export_visuals(export_dir=target_folder, file_name="temp_result")

                # 5. Rename to original filename
                generated_file = os.path.join(target_folder, "temp_result.png")
                final_path = os.path.join(target_folder, filename)

                if os.path.exists(generated_file):
                    if os.path.exists(final_path):
                        os.remove(final_path)
                    os.rename(generated_file, final_path)
                    print("Saved.")

                    total_count += 1
                    folder_image_count += 1 # Increment folder counter
                else:
                    print("No detection (or save failed).")

            except Exception as e:
                print(f"ERROR: {e}")
                errors += 1

print("\n" + "="*40)
print(f"Done! Processed {total_count} images.")
print(f"Errors encountered: {errors}")
print(f"All results saved to: {OUTPUT_ROOT}")
print("="*40)

Loading model from /content/runs/detect/train/weights/best.onnx...
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Processing images from /content/dataset/test (Max 10 per folder)...
[1] Processing: Andres - Outdoor - 12C.png ... Performing prediction on 135 slices.
Loading /content/runs/detect/train/weights/best.onnx for ONNX Runtime inference...
Using ONNX Runtime 1.23.2 ('CUDAExecutionProvider', {'device_id': 0})
Saved.
[2] Processing: Andres - Outdoor - 15C.png ... Performing prediction on 135 slices.
Saved.
[3] Processing: Andres - Outdoor - 18C.png ... Performing prediction on

In [6]:
!zip -r /content/inference_results.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/Outdoor/ (stored 0%)
  adding: content/results/Outdoor/Non-masked/ (stored 0%)
  adding: content/results/Outdoor/Non-masked/Andres - Outdoor - 24C.png (deflated 3%)
  adding: content/results/Outdoor/Non-masked/Pablo - Outdoor - 6C.png (deflated 4%)
  adding: content/results/Outdoor/Non-masked/Cristina - Outdoor - 12C.png (deflated 4%)
  adding: content/results/Outdoor/Non-masked/Narciso - Outdoor - 6C.png (deflated 4%)
  adding: content/results/Outdoor/Non-masked/Narciso - Outdoor - 12C.png (deflated 5%)
  adding: content/results/Outdoor/Non-masked/DaniF - Outdoor - 24C.png (deflated 6%)
  adding: content/results/Outdoor/Non-masked/Andres - Outdoor - 15C.png (deflated 4%)
  adding: content/results/Outdoor/Non-masked/Isa - Outdoor - 21C.png (deflated 8%)
  adding: content/results/Outdoor/Non-masked/Andres - Outdoor - 9C.png (deflated 4%)
  adding: content/results/Outdoor/Non-masked/Andres - Outdoor - 21C.png (deflated 3%)


In [7]:
!cp -r /content/runs /content/drive/MyDrive/runs_face_detect

In [8]:
!cp -r /content/results /content/drive/MyDrive/results_face_detect

In [10]:
!zip -r /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/predict2/ (stored 0%)
  adding: content/runs/detect/train/ (stored 0%)
  adding: content/runs/detect/train/val_batch2_labels.jpg (deflated 7%)
  adding: content/runs/detect/train/train_batch2.jpg (deflated 6%)
  adding: content/runs/detect/train/args.yaml (deflated 53%)
  adding: content/runs/detect/train/confusion_matrix.png (deflated 30%)
  adding: content/runs/detect/train/labels.jpg (deflated 32%)
  adding: content/runs/detect/train/train_batch7980.jpg (deflated 11%)
  adding: content/runs/detect/train/results.csv (deflated 63%)
  adding: content/runs/detect/train/results.png (deflated 8%)
  adding: content/runs/detect/train/val_batch2_pred.jpg (deflated 7%)
  adding: content/runs/detect/train/BoxR_curve.png (deflated 12%)
  adding: content/runs/detect/train/confusion_matrix_normalized.png (deflated 29%)
  adding: content/runs/detect/train/val_batch1_pred.jpg (deflated 8%)
 

In [ ]:
!cp

#Face recognition

In [9]:
!pip install insightface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 32.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071350 sha256=f345c3652197a8f53249dcb164c334f13ce03e7b4a1c4e95eeead3e24d102d04
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [ ]:
import os
import cv2
import pickle
import numpy as np
from insightface.app import FaceAnalysis

# --- CONFIGURATION ---
KNOWN_FACES_DIR = '/content/dataset/db'  # Folder with reference images
DB_PATH = '/content/face_database.pkl'     # Where to save the DB

# 1. Initialize ArcFace
print("Loading ArcFace model...")
app = FaceAnalysis(name='buffalo_l', providers=['CUDAExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))

# Dictionary to store list of embeddings per person
# Format: { 'Andres': [emb1, emb2, ...], 'Maria': [emb1, ...] }
temp_db = {}

# 2. Walk through all subfolders
if not os.path.exists(KNOWN_FACES_DIR):
    os.makedirs(KNOWN_FACES_DIR)
    print(f"Created {KNOWN_FACES_DIR}. Please upload folders/images there!")
else:
    print(f"Scanning {KNOWN_FACES_DIR} recursively...")

    for root, dirs, files in os.walk(KNOWN_FACES_DIR):
        for filename in files:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):

                # --- PARSING LOGIC ---
                # filename: "Andres - Outdoor.jpg"
                # 1. Remove extension -> "Andres - Outdoor"
                base_name = os.path.splitext(filename)[0]

                # 2. Split by " - " and take the first part
                # If filename is just "Andres.jpg", it returns "Andres"
                person_name = base_name.split(' - ')[0].strip()

                img_path = os.path.join(root, filename)
                img = cv2.imread(img_path)
                if img is None: continue

                # Detect and Extract
                # We use the full image here. If the image has multiple people,
                # we assume the largest face is the target user.
                faces = app.get(img)

                if len(faces) > 0:
                    # Get the embedding (512-dimensional vector)
                    # We create a new list for this person if they are new
                    if person_name not in temp_db:
                        temp_db[person_name] = []

                    # Sort faces by size (width * height) to pick the main person
                    faces.sort(key=lambda x: (x.bbox[2]-x.bbox[0]) * (x.bbox[3]-x.bbox[1]), reverse=True)

                    # Store the embedding
                    temp_db[person_name].append(faces[0].embedding)
                    print(f"Added feature for: {person_name} (from {filename})")
                else:
                    print(f"Warning: No face found in {filename}")

    # 3. Consolidate (Average) Embeddings
    # If we have multiple photos of Andres, we average them to create a robust ID
    final_db = {}

    print("\nConsolidating database...")
    for name, embeddings in temp_db.items():
        if len(embeddings) > 1:
            # Calculate mean
            mean_embedding = np.mean(embeddings, axis=0)
            # Normalize (ArcFace requires unit vectors)
            final_embedding = mean_embedding / np.linalg.norm(mean_embedding)
            print(f" -> {name}: Combined {len(embeddings)} images.")
        else:
            # Just normalize the single embedding
            final_embedding = embeddings[0] / np.linalg.norm(embeddings[0])
            print(f" -> {name}: Single reference image.")

        final_db[name] = final_embedding

    # 4. Save
    with open(DB_PATH, 'wb') as f:
        pickle.dump(final_db, f)
    print(f"\nDatabase saved to {DB_PATH}. Total Identities: {len(final_db)}")